#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [7]:
model = Sequential()

In [8]:
#卷積層1

In [9]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))

In [10]:
model.add(Dropout(rate=0.25))

In [11]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
#卷積層2與池化層2

In [13]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))

In [14]:
model.add(Dropout(0.25))

In [15]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [17]:
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [18]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))

In [19]:
model.add(Dense(10, activation='softmax'))

In [20]:
#檢查model 的STACK
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)             

# 載入之前訓練的模型

In [21]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

### loss = 'categorical_crossentropy'

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''

train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=10, batch_size=128, verbose=1) 

Train on 37500 samples, validate on 12500 samples
Epoch 1/10
37500/37500 [==============================] - 128s 3ms/step - loss: 2.1653 - accuracy: 0.2021 - val_loss: 2.0543 - val_accuracy: 0.2972
Epoch 2/10
37500/37500 [==============================] - 130s 3ms/step - loss: 1.9510 - accuracy: 0.2996 - val_loss: 1.9525 - val_accuracy: 0.3371
Epoch 3/10
37500/37500 [==============================] - 128s 3ms/step - loss: 1.8250 - accuracy: 0.3502 - val_loss: 1.8228 - val_accuracy: 0.3962
Epoch 4/10
37500/37500 [==============================] - 132s 4ms/step - loss: 1.7224 - accuracy: 0.3866 - val_loss: 1.7424 - val_accuracy: 0.4193
Epoch 5/10
37500/37500 [==============================] - 129s 3ms/step - loss: 1.6387 - accuracy: 0.4162 - val_loss: 1.6557 - val_accuracy: 0.4543
Epoch 6/10
37500/37500 [==============================] - 128s 3ms/step - loss: 1.5689 - accuracy: 0.4368 - val_loss: 1.6094 - val_accuracy: 0.4692
Epoch 7/10
37500/37500 [==============================] - 128s

In [23]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print("loss: categorical_crossentropy")
print('accuracy=',scores[1])
print('loss=',scores[0])

10000/10000 [==============================] - 13s 1ms/step
loss: categorical_crossentropy
accuracy= 0.5306000113487244
loss= 1.4215528938293458


### loss = 'MSE'

In [24]:
model.compile(loss='MSE', optimizer='sgd', metrics=['accuracy'])

'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''

train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=10, batch_size=128, verbose=1) 

Train on 37500 samples, validate on 12500 samples
Epoch 1/10
37500/37500 [==============================] - 131s 4ms/step - loss: 0.0625 - accuracy: 0.5108 - val_loss: 0.0649 - val_accuracy: 0.5279
Epoch 2/10
37500/37500 [==============================] - 130s 3ms/step - loss: 0.0623 - accuracy: 0.5150 - val_loss: 0.0648 - val_accuracy: 0.5287
Epoch 3/10
37500/37500 [==============================] - 131s 3ms/step - loss: 0.0624 - accuracy: 0.5144 - val_loss: 0.0646 - val_accuracy: 0.5298
Epoch 4/10
37500/37500 [==============================] - 132s 4ms/step - loss: 0.0623 - accuracy: 0.5153 - val_loss: 0.0646 - val_accuracy: 0.5306
Epoch 5/10
37500/37500 [==============================] - 131s 4ms/step - loss: 0.0622 - accuracy: 0.5163 - val_loss: 0.0645 - val_accuracy: 0.5282
Epoch 6/10
37500/37500 [==============================] - 131s 3ms/step - loss: 0.0621 - accuracy: 0.5155 - val_loss: 0.0645 - val_accuracy: 0.5324
Epoch 7/10
37500/37500 [==============================] - 132s

In [25]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print("loss: MSE")
print('accuracy=',scores[1])
print('loss=',scores[0])

10000/10000 [==============================] - 13s 1ms/step
loss: MSE
accuracy= 0.5351999998092651
loss= 0.06391070890426635


### loss = 'binary_crossentropy'

In [26]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''

train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=10, batch_size=128, verbose=1) 

Train on 37500 samples, validate on 12500 samples
Epoch 1/10
37500/37500 [==============================] - 133s 4ms/step - loss: 0.2093 - accuracy: 0.9179 - val_loss: 0.2189 - val_accuracy: 0.9127
Epoch 2/10
37500/37500 [==============================] - 132s 4ms/step - loss: 0.2088 - accuracy: 0.9183 - val_loss: 0.2178 - val_accuracy: 0.9131
Epoch 3/10
37500/37500 [==============================] - 133s 4ms/step - loss: 0.2092 - accuracy: 0.9181 - val_loss: 0.2176 - val_accuracy: 0.9128
Epoch 4/10
37500/37500 [==============================] - 133s 4ms/step - loss: 0.2077 - accuracy: 0.9184 - val_loss: 0.2174 - val_accuracy: 0.9133
Epoch 5/10
37500/37500 [==============================] - 132s 4ms/step - loss: 0.2073 - accuracy: 0.9185 - val_loss: 0.2165 - val_accuracy: 0.9135
Epoch 6/10
37500/37500 [==============================] - 133s 4ms/step - loss: 0.2067 - accuracy: 0.9187 - val_loss: 0.2152 - val_accuracy: 0.9142
Epoch 7/10
37500/37500 [==============================] - 132s

In [27]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print("loss: binary_crossentropy")
print('accuracy=',scores[1])
print('loss=',scores[0])

10000/10000 [==============================] - 13s 1ms/step
loss: binary_crossentropy
accuracy= 0.9140701293945312
loss= 0.21315780186653138
